In [243]:
# Import 
import pandas as pd
# To install, pip install tabula-py in your env
import tabula
from sqlalchemy import create_engine
import psycopg2
from config import db_password

In [244]:
tabula.convert_into(r"resources\2020_data.pdf", r"uncleaned_csv\adult_access2020.csv", output_format="csv", pages='34', stream = True)

In [245]:
# Read in csv
adult_access_df = pd.read_csv(r"uncleaned_csv\adult_access2020.csv")
adult_access_df
adult_access_df = adult_access_df.dropna(how='all')

In [246]:
# Extract second half of ami_df to make second df to be joined back in
adult_access_df_2 = adult_access_df[["Rank", "State", "%.1", "#.1"]]
adult_access_df_2 = adult_access_df_2.rename(columns={"%.1" : "%", "#.1": "#"})


In [247]:
# drop seperated columns
adult_access_df = adult_access_df.drop(columns={"Rank", "State", "%.1", "#.1"})

In [248]:
# Splitup problematic data
adult_access_df['Rank'], adult_access_df['State'] = adult_access_df['Rank State'].str.split(' ', 1).str

adult_access_df = adult_access_df.drop(columns='Rank State')

C:\Users\noaht\AppData\Local\Temp/ipykernel_1772/1540202657.py:2: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  adult_access_df['Rank'], adult_access_df['State'] = adult_access_df['Rank State'].str.split(' ', 1).str


In [249]:
# Rename and reappend cleaned half of dataframe.
adult_access_df = adult_access_df[['Rank', 'State', '%', '#']]
adult_access_df = adult_access_df.append(adult_access_df_2)

In [250]:
# Reset index to make cleaning possible
adult_access_df = adult_access_df.reset_index(drop=True)

In [251]:
# Assign proper values
adult_access_df.loc[15, "Rank"]=16
adult_access_df.loc[15, "State"]='Nebraska'
adult_access_df.loc[41, "#"]=829000

In [252]:
# change dtype of # column
adult_access_df['#'] = adult_access_df['#'].str.replace(',', '').astype(float)

In [253]:
# Add suffix 
adult_access_df = adult_access_df.add_suffix('_adult_care_access_2020')

In [254]:
adult_access_df = adult_access_df.rename(columns={'State_adult_care_access_2020' : 'State'})

In [255]:
adult_access_df

,Rank_adult_care_access_2020,State,%_adult_care_access_2020,#_adult_care_access_2020
0,1,Vermont,40.7,42000.0
1,2,New Hampshire,44.6,89000.0
2,3,Rhode Island,45.7,73000.0
3,4,Iowa,47.3,214000.0
4,5,Delaware,47.5,64000.0
5,6,Colorado,48.7,407000.0
6,7,Minnesota,49.5,360000.0
7,8,Maine,49.6,99000.0
8,9,Missouri,50.9,485000.0
9,10,Michigan,51.1,713000.0


In [202]:
adult_access_df.to_csv(r'resources\adult_access2020.csv', sep=',', index=False)

In [213]:
"postgresql://[user]:[password]@[location]:[port]/[database]"


In [235]:
pg_user = "postgres"
pg_pwd = db_password
pg_port = "5432"

In [236]:
host = 'project-vu-database-piecharts.c7rvpt2rehpr.us-east-2.rds.amazonaws.com'

In [240]:
db_string = "postgresql://{username}:{password}@{host}:{port}/project_db".format(username=pg_user, password=pg_pwd, host = host, port=pg_port)


In [241]:
engine = create_engine(db_string)


In [242]:
adult_access_df.to_sql(name='adult_access_care2020', con=engine, index=False)

In [ ]:
# # Do not run this yet!
# for data in pd.read_csv(fr'resources\adult_access2020.csv', chunksize=1000000):
#     data.to_sql(name='adult_care_access_2020', con=engine, if_exists='append')